# CAPSTONE PROJECT - 8

### Domain: 
#### Financial Services
### Title:
#### Exploratory Data Analysis and Credit Assessment in Financial Services

### About:
#### Finance is a field that is concerned with the allocation (investment) of assets and liabilities over space and time, often under conditions of risk or uncertainty. Finance can also be defined as the art of money management. Participants in the market aim to price assets based on their risk level, fundamental value, and their expected rate of return.
#### The dataset consists of customer information on L&T financial services. It is a finance dataset, which consists of customers’ demographics, loans disbursed, asset cost being purchased, and the customers’ previous accounts and loan histories. The dataset also consists of the state and branch id of L&T from where the loan was disbursed and the customer’s account history. It also contains the CNS score and score description provided by the Credit Bureaus of India.
#### It is a challenge for any financial service to target the right people for disbursing the loan. The credit team must analyze various details like CIBIL score, payment history (if available), credit history, geographical location, profession, income, age, education, etc. of the customers. This will help in understanding whether the person is capable of paying back the loan amount. Which in turn reduces its NPAs and increases its profitability.
#### You need to assess what data is available and perform some exploratory and descriptive analytics to identify interesting and useful patterns, trends, and insights.


## CheckPoint 1

#### Task 1.1 Data manipulation and Visualization using Python
#### Task 1.2 Exploratory Data Analysis & Statistical Analysis

## Task 1.1 Data manipulation and Visulization using Python

### Step A & B
- Load the dataset: Import the dataset into a Python environment (e.g., using pandas library) and create a data frame.

- Data exploration: Perform initial exploration of the dataset to gain insights into its structure and content. Use functions such as .head(), .info(), .describe(), and .shape to understand the data's dimensions, variable types, and summary statistics.

#### Importing the Packages

In [1]:
! pip install openpyxl

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px
import scipy.stats as st
from scipy import stats
import seaborn as sns

from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')

#### Loading the datasets into dataframe

In [13]:
loan_details = pd.read_csv("Loan_Details_transactions.csv")
df2 = pd.read_csv("Loan_status_mapping.csv")
branch_id = pd.read_csv("Branch_ID_Master.csv")
postal_code = pd.read_excel("Postal_Code_Master.xlsx" )
city_master = pd.read_excel("City_Master.xlsx")
region_master = pd.read_excel("Region_Master.xlsx")
state_master = pd.read_excel("State_Master.xlsx")
state_region = pd.read_excel("State_Region_Mapping.xlsx")
df3 = df2.drop(['Loan_Id'],axis = 1)

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
merged_df = pd.merge(loan_details, city_master, on=['City_Code', 'State_Code'], how='left')
merged_df1 = pd.merge(merged_df,state_master, on=['State_Code'], how='left')
merged_df2 = pd.merge(merged_df1,branch_id, on=['Branch_Id'], how='left')
merged_df3 = pd.merge(merged_df2,state_region[['Region_ID','State_Code']], on=['State_Code'], how='left')

loan_details = pd.concat([merged_df3,df3],axis=1)
loan_details

## Data Exploration

In [ ]:
loan_details.describe()

In [ ]:
loan_details.info()

In [ ]:
loan_details.shape

#### All the numeric data columns

In [ ]:
numeric_type = loan_details.select_dtypes(include=['int64','float64']).columns
print(pd.DataFrame(numeric_type))

#### All the catagorical data columns

In [ ]:
objects_type = loan_details.select_dtypes(include='object').columns
print(pd.DataFrame(objects_type))

## Data Cleaning
#### Checking for any duplicate values

In [ ]:
loan_details[loan_details.duplicated].sum()

#### Checking for missing values under each column

In [ ]:
loan_details.isnull().sum()

In [ ]:
# Total missing values
loan_details.isnull().sum().sum()

In [ ]:
# Overall Percentage of missing values 
(loan_details.isnull().sum().sum()/len(loan_details))*100

In [ ]:
# Replacing the missing value with the mode 
loan_details['Employment_Type'].fillna(loan_details['Employment_Type'].mode()[0],inplace=True)
loan_details['Region_ID'].fillna(loan_details['Region_ID'].mode()[0],inplace=True)

In [ ]:
loan_details.isnull().sum()

## Univariate Analysis

In [ ]:
univariate = loan_details.drop(['Loan_Id','Disbursed_Amount','Asset_Cost','ltv','Date_of_Birth','DisbursalDate',
         'CREDIT.HISTORY.LENGTH', 'PERFORM_CNS.SCORE'],axis = 1)
univariate.hist(figsize=(20,10),color = 'orange')
plt.show()

## Bivariate Analysis

In [ ]:
# constructing scatter plot for disbursed amount and asset cost

plt.scatter(loan_details['Disbursed_Amount'] , loan_details['Asset_Cost'],linewidths = 0.71,
            marker ="s",edgecolor ="orange",s = 1)
plt.show()

In [ ]:
var =pd.crosstab(loan_details['Employment_Type'], loan_details['Loan_Default'])
var.div(var.sum(0).astype(float),axis=0).plot(kind="bar",stacked=True )

In [ ]:
var =pd.crosstab(loan_details['Aadhar_flag'], loan_details['Loan_Default'])
var.div(var.sum(0).astype(float),axis=1).plot(kind="bar",stacked=True )

In [ ]:
fig, ax = plt.subplots(figsize=(50, 15))
sns.boxplot(loan_details)

## Feature Engineering

#### Outlier detection and handling (Disbursed_Amount)

In [ ]:
sns.boxplot(loan_details['Disbursed_Amount'])

In [ ]:
# Finding number of outliers using the interquantile range

q1 = np.quantile(loan_details['Disbursed_Amount'],0.25)
q3 = np.quantile(loan_details['Disbursed_Amount'],0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr)
print(lower_bound)
higher_bound = q3 + (1.5* iqr)
print(higher_bound)

median = loan_details['Disbursed_Amount'].median()
print("Median: ",median)
loan_details[(loan_details['Disbursed_Amount'] > higher_bound) | (loan_details['Disbursed_Amount'] < lower_bound)]

In [ ]:
def replace(data,median):
    cleaned_data = [median if val < lower_bound or val > higher_bound else val for val in data]
    return cleaned_data

In [ ]:
new_disbursed = pd.DataFrame(replace(loan_details['Disbursed_Amount'] , median))
new_disbursed

In [ ]:
del loan_details['Disbursed_Amount']

In [ ]:
loan_details.insert(1, 'Disbursed_Amount',new_disbursed)

In [ ]:
sns.boxplot(loan_details['Disbursed_Amount'])

#### Outlier detection and handling (Asset_cost)

In [ ]:
q1 = np.quantile(loan_details['Asset_Cost'],0.25)
q3 = np.quantile(loan_details['Asset_Cost'],0.75)
iqr = q3 - q1
lower_asset = q1 - (1.5 * iqr)
print(lower_asset)
higher_asset = q3 + (1.5* iqr)
print(higher_asset)
print(loan_details['Asset_Cost'].median())

In [ ]:
def replace(data,median):
    cleaned_data = [median if val < lower_asset or val > higher_asset else val for val in data]
    return cleaned_data

new_asset = pd.DataFrame(replace(loan_details['Asset_Cost'] , median))

In [ ]:
del loan_details['Asset_Cost']

In [ ]:
loan_details.insert(2, 'Asset_Cost',new_asset)

In [ ]:
sns.boxplot(loan_details['Asset_Cost'])

#### Changing the date to month in CREDIT.HISTORY.LENGTH

In [ ]:
def convert(i):
    parts = i.split()
    year = ''
    month = ''
    for j in parts[0]:
        if j == '0' or j == '1' or j == '2' or j == '3' or j == '4' or j == '5' or j == '6' or j == '7' or j == '8' or j == '9':
            year = year +j
    year = int(year)
    for j in parts[1]:
        if j == '0' or j == '1' or j == '2' or j == '3' or j == '4' or j == '5' or j == '6' or j == '7' or j == '8' or j == '9':
            month =month + j
    month = int(month)
    return (year * 12) + month
data1 = loan_details['CREDIT.HISTORY.LENGTH'].tolist()
months = [convert(i) for i in data1]
new = pd.DataFrame(months)

loan_details.drop(['CREDIT.HISTORY.LENGTH'] , axis = 1 ,inplace = True)
loan_details.insert(16, 'CREDIT.HISTORY.LENGTH',new)

#### Changing the Date_of_Birth into year

In [ ]:
loan_details['Date_of_Birth']=pd.to_datetime(loan_details.Date_of_Birth)
loan_details['Date_of_Birth']=loan_details['Date_of_Birth'].dt.year
loan_details=loan_details.drop('DisbursalDate',axis=1)

In [ ]:
loan_details['Region_ID'].fillna(loan_details['Region_ID'].mode()[0],inplace=True)

## Label encoding

In [ ]:
columns = loan_details.columns
object_columns = []
for column in columns:
    if(loan_details[column].dtype == 'object'):
        object_columns.append(column)
        
binary_columns = []
odinary_columns = []
for column in object_columns:
    if(loan_details[column].unique().all() in ['yes','no']):
        binary_columns.append(column)
    else:
        odinary_columns.append(column)

In [ ]:
encoder = OrdinalEncoder()
loan_details[odinary_columns] = encoder.fit_transform(loan_details[odinary_columns])

In [ ]:
loan_details.info()

In [ ]:
loan_details=loan_details.drop(['MobileNo_Avl_Flag'],axis=1)

## Correlation

In [ ]:
corrs = loan_details.corr()
corrs

In [ ]:
matrix = loan_details.corr()
plt.subplots(figsize=(20,22))
sns.heatmap(matrix,vmax=.8,square=True,cmap='coolwarm', annot = True)

## CheckPoint 2

#### Task 2.1 (Visualization using Power-BI Dashboard)
#### Task 2.2 (Model building using ML algorithms)

## Hypothethis Testing

#### Hypothesis 1 : The average credit history length of borrowers who own a passport is significantly different from those who don't have

In [ ]:
loan_details['Passport_flag'].value_counts()

In [ ]:
credit_history_passport = loan_details[loan_details['Passport_flag'] == 1]['CREDIT.HISTORY.LENGTH']
credit_history_passport.shape

In [ ]:
credit_history_no_passport = loan_details[loan_details['Passport_flag'] == 0]['CREDIT.HISTORY.LENGTH']
credit_history_no_passport.shape

In [ ]:
## First we will calculate mean and std. deviation for the customer havng passport
mean_credit_history_passport = credit_history_passport.mean()
std_credit_history_passport = credit_history_passport.std()

## Then we will calculate the mean and std. deviation of customer not having passport
mean_credit_history_no_passport = credit_history_no_passport.mean()
std_credit_history_no_passport = credit_history_no_passport.std()

In [ ]:
# We perform the t-test for independent samples
t_stat, p_value = stats.ttest_ind(credit_history_passport, credit_history_no_passport, equal_var=False)

# We determine the significance of the test and interpret the results
alpha = 0.05 

In [ ]:
print(f"Mean Credit History Length for Borrowers with Passport: {mean_credit_history_passport:.2f}")
print(f"Mean Credit History Length for Borrowers without Passport: {mean_credit_history_no_passport:.2f}")
print(f"T-statistic: {t_stat:.2f}")
print(f"P-value: {p_value:.4f}")

#### Checking the hypothesis

In [ ]:
if p_value < alpha:
    print("The p-value is less than the significance level, Reject the null hypothesis.")
else:
    print("The p-value is greater than the significance level, Fail to reject the null hypothesis.")

#### Hypothesis 2 : There is a significant difference in the mean ltv(loan to value ratio) between Self-employed borrowers and Salaried borrowers

In [ ]:
loan_details['Employment_Type'].value_counts()

In [ ]:
ltv_self_employed = loan_details[loan_details['Employment_Type'] == 0]['ltv']
ltv_self_employed.shape

In [ ]:
ltv_salaried = loan_details[loan_details['Employment_Type'] == 1]['ltv']
ltv_salaried.shape

In [ ]:
mean_ltv_self_employed = ltv_self_employed.mean()
std_ltv_self_employed = ltv_self_employed.std()

In [ ]:
mean_ltv_salaried = ltv_salaried.mean()
std_ltv_salaried = ltv_salaried.std()

In [ ]:
t_stat, p_value = stats.ttest_ind(ltv_self_employed, ltv_salaried, equal_var=False)
alpha = 0.05

In [ ]:
print(f"Mean ltv for Self-employed: {mean_ltv_self_employed:.2f}")
print(f"Mean ltv for Salaried: {mean_ltv_salaried:.2f}")
print(f"T-statistic: {t_stat:.2f}")
print(f"P-value: {p_value:.4f}")

#### Checking the hypothethis

In [ ]:
if p_value < alpha:
    print("The p-value is less than the significance level . Reject the null hypothesis.")
else:
    print("The p-value is greater than the significance level . Fail to reject the null hypothesis.")

## Model Creation and Model Evaluation

In [ ]:
# Segregation the data in two parts X and y
new_loan_details = loan_details
X = new_loan_details.drop(columns=['Loan_Default'],axis=1)   # independent variable
y = new_loan_details['Loan_Default']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42,stratify=y)

#### Logistic Regression Model

In [ ]:
# Create a logistic regression model
model = LogisticRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score (X_train_prediction,y_train)
print('Accuracy score on training data : ',training_data_accuracy)

In [ ]:
# accuracy score on test data
X_test_prediction = model.predict(X_test)
accuracy = accuracy_score (X_test_prediction,y_test)
print('Accuracy score on test data:',accuracy)

#### Logistic Regression Evaluation

In [ ]:
cnf_matrix=confusion_matrix(y_test, X_test_prediction)
cnf_matrix

In [ ]:
print(classification_report(y_test, X_test_prediction))

In [ ]:
TN,FP,FN,TP=cnf_matrix.ravel()
print(TN,FP,FN,TP)
linear_test_data_accuracy=accuracy_score(y_test, X_test_prediction)
print('Accuracy_score',linear_test_data_accuracy )

#### K-Fold

In [ ]:
# Initialize KFold with the desired number of splits
kf = KFold(n_splits=10, shuffle=True, random_state=42)
smote=SMOTE(sampling_strategy='auto',random_state=42)
LR = LogisticRegression()

# perform k-fold cross-validation using  logistic regression
LR_metric=cross_val_score(LR,X,y,cv=kf)
print(LR_metric)

# calculate the average performance
kFold_accuracy=LR_metric.mean()*100.0
print("Accuracy: ",kFold_accuracy)

#### Smote

In [ ]:
X = new_loan_details.drop(columns=['Loan_Default'],axis=1)   # independent variable
y = new_loan_details['Loan_Default']

In [ ]:
# Generate a synthetic imbalanced dataset
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print('Before applying smote method: ',X_train.shape)
# Apply SMOTE to balance the classes in the training set
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
print('After applying smote method: ',X_train_resampled.shape)
# Train a model on the resampled training data
model = LogisticRegression()
model.fit(X_train_resampled, y_train_resampled)
# Make predictions on the test set
y_pred = model.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
cnf_matrix=confusion_matrix(y_test, y_pred)
cnf_matrix

#### Smote Evaluation

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
TN,FP,FN,TP=cnf_matrix.ravel()
print(TN,FP,FN,TP)
smote_Accuracy = accuracy_score(y_test, y_pred)
print('Accuracy_score: ',smote_Accuracy )

## Result

In [ ]:
print('Accuracy by Linear Regression: ',round(linear_test_data_accuracy*100,3))
print('Accuracy by Linear Regression with smote : ',round(smote_Accuracy*100,3))
print('Accuracy by KFold: ',round(kFold_accuracy,3))